In [1]:
#IMPORTAMOS LAS LIBRERIAS QUE SE USARAN EN EL ETL
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import json
import ast

In [2]:
#cargamos el dataset .csv 
movies_clean=pd.read_csv('datasets/movies_clean.csv')

In [3]:
movies_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  float64
 2   genres                 42992 non-null  object 
 3   id                     45376 non-null  int64  
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  float64
 7   production_companies   33580 non-null  object 
 8   production_countries   39165 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       41608 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

*FUNCIONES SOLICITADAS PARA EL API

In [4]:
#funcion de consulta de películas por mes
def peliculas_mes(mes:str):

    mes = mes.lower()
    meses = {'enero': 1,'febrero': 2,'marzo': 3,'abril': 4,'mayo': 5,'junio': 6,'julio': 7,'agosto': 8,'septiembre': 9,'octubre': 10,'noviembre': 11,'diciembre': 12}
    mes_numero = meses[mes]

    # Convertir la columna "fecha" a un objeto de tipo fecha
    movies_clean['release_date'] = pd.to_datetime(movies_clean['release_date'])

    try:
        month_filtered = movies_clean[movies_clean['release_date'].dt.month == mes_numero]
    except (ValueError, KeyError, TypeError):
        return None

    month_unique = month_filtered.drop_duplicates(subset='id')
    respuesta = month_unique.shape[0]

    return {'mes':mes, 'cantidad':respuesta}

In [5]:
peliculas_mes('febrero')

{'mes': 'febrero', 'cantidad': 3028}

In [6]:
# Funcion de consulta de peliculas por dia de la semana 
def peliculas_dia(dia:str):

    days = {'lunes': 'Monday', 'martes': 'Tuesday','miercoles': 'Wednesday','jueves': 'Thursday','viernes': 'Friday','sabado': 'Saturday','domingo': 'Sunday'}
    day = days[dia.lower()]
    lista_peliculas_day = movies_clean[movies_clean['release_date'].dt.day_name() == day].drop_duplicates(subset='id')
    cantidad = lista_peliculas_day.shape[0]

    return {'dia': dia, 'cantidad': cantidad}

In [7]:
peliculas_dia('miercoles')

{'dia': 'miercoles', 'cantidad': 7027}

In [8]:
# # Funcion de consulta del numero de colecciones de peliculas, su ganancia total y ganancia promedio  otro metodo
def franquicia(franquicia):
    movies_clean['belongs_to_collection']=movies_clean['belongs_to_collection'].fillna("")
    #lista_pelis_franquicia = movies_clean[(movies_clean['belongs_to_collection'] == 'franquicia')].drop_duplicates(subset='id')
    franquicia_df=movies_clean[movies_clean['belongs_to_collection'].apply (lambda x: franquicia in str(x))]
    #para jalar la cantidad e peliculas, ganancia total 
    cantidad_pelis = len(franquicia_df)
    revenue_franq = franquicia_df['revenue'].sum()
    promedio_franq = franquicia_df['revenue'].mean()
    return {'franquicia':franquicia, 'cantidad':cantidad_pelis, 'ganancia_total':revenue_franq, 'ganancia_promedio':promedio_franq}

In [9]:
franquicia('Toy Story Collection')

{'franquicia': 'Toy Story Collection',
 'cantidad': 3,
 'ganancia_total': 1937890605.0,
 'ganancia_promedio': 645963535.0}

In [10]:
# 4. Funcion de consulta del numero de peliculas por pais 
def peliculas_pais(pais):
    # Selecciona todas las películas del DataFrame 'movies_clean' cuya columna 'production_country' contiene el país especificado.
    # La función lambda se utiliza para aplicar la operación a cada valor de la columna 'production_country'.
    peliculas_pais = movies_clean[movies_clean['production_countries'].apply(lambda x: pais in str(x) if pd.notnull(x) else False)]
    
    # Elimina las filas duplicadas de la película para evitar contar varias veces una misma película.
    peliculas_pais = peliculas_pais.drop_duplicates(subset='id')
    
    # Cuenta la cantidad de películas restantes después de eliminar los duplicados.
    cantidad_peliculas = len(peliculas_pais)
    
    # Devuelve los resultados en un diccionario con claves legibles.
    return {'pais': pais, 'cantidad': cantidad_peliculas}


In [11]:
peliculas_pais('United States of America')

{'pais': 'United States of America', 'cantidad': 21134}

In [12]:
# 5. Funcion de consulta del numero de peliculas por productora, ganancias totales y numero de peliculas
def productoras(productora):
    movies_clean['production_companies']=movies_clean['production_companies'].fillna("")
    productora_df=movies_clean[movies_clean['production_companies'].apply(lambda x: productora in str(x))]

    ganancia_total= productora_df['revenue'].sum()
    cantidad_pelis_prod= len(productora_df)

    return {'productora':productora, 'ganancia_total':ganancia_total, 'cantidad':cantidad_pelis_prod}

In [13]:
productoras('Pixar Animation Studios')

{'productora': 'Pixar Animation Studios',
 'ganancia_total': 11188533734.0,
 'cantidad': 52}

In [14]:
#6.  Funcion de consulta de peliculas, su inversion, ganancias, retorno y año
def retorno(pelicula):
    info_pelicula = movies_clean[(movies_clean['title'] == pelicula)].drop_duplicates(subset='title')
    pelicula_nombre = info_pelicula['title'].iloc[0]
    inversion_pelicula = str(info_pelicula['budget'].iloc[0])
    ganancia_pelicula = str(info_pelicula['revenue'].iloc[0])
    retorno_pelicula = str(info_pelicula['return'].iloc[0])
    year_pelicula = str(info_pelicula['release_year'].iloc[0])

    return {'pelicula':pelicula_nombre, 'inversion':inversion_pelicula, 'ganacia':ganancia_pelicula,'retorno':retorno_pelicula, 'anio':year_pelicula}

In [15]:
retorno('Jumanji')

{'pelicula': 'Jumanji',
 'inversion': '65000000.0',
 'ganacia': '262797249.0',
 'retorno': '4.0430346',
 'anio': '1995'}

In [16]:
data=pd.read_csv('datasets/movies_dataset.csv')

C:\Users\CARRILLO JARA\AppData\Local\Temp\ipykernel_18012\2963903898.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('datasets/movies_dataset.csv')


In [17]:
data = data.drop_duplicates(subset = 'title')
C = data['vote_average'].mean()
m = data['vote_count'].quantile(0.90)
data = data.loc[data['vote_count'] >= m]
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)
data['score'] = data.apply(weighted_rating, axis=1)
data = data.sort_values('score', ascending = False)
tfidf = TfidfVectorizer(stop_words='english')
data['overview'] = data['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['overview'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
data.reset_index(drop = True, inplace = True)
index = pd.Series(data.index, index = data['title']).drop_duplicates()

In [18]:
# 7 recomendacion 
def recomendacion(titulo:str):  
    local_cosine_sim = cosine_sim
    if titulo not in index:
        return "La película no se encuentra entra en la Base de Datos, Intenta con otra!"

    idx = index[titulo]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    result = data['title'].iloc[movie_indices]
    return {"lista recomendada" : list(result)}

In [19]:
recomendacion('Avatar')

{'lista recomendada': ['Apollo 18',
  'The American',
  'The Matrix',
  'Europa Report',
  'A Grand Day Out']}